## Create Model Geometry
#### use Gmsh to create geometry and mesh for Wenchuan fault
#### created by D. Li
#### 16 Mar. 2024

In [2]:
# load model
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

import os
import sys
import math
import pyproj

%matplotlib notebook

In [3]:
# load data
# change format to CSV file: bash change_format.sh

rootfolder = '/Users/shihao/Documents/SeisSol/wenchuan/'
xyzfile = rootfolder + 'CDYingxiu.csv'

faultData = pd.read_csv(xyzfile)

# print(faultData)

# converge coordinages

z = faultData["z"].to_numpy()
lon = faultData["longitude"].to_numpy()
lat = faultData["latitude"].to_numpy()

dip  = faultData['dip'].to_numpy()
strik  =faultData['azimuth'].to_numpy()

# convert  # UTM projection
lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
myproj = pyproj.Proj(init='EPSG:32648', datum='WGS84') # projection code UTM 48N

xflt,yflt = pyproj.transform(lla, myproj, lon,lat, radians=False)

print(z, xflt,yflt)
print(z.min(),z.max(),xflt.min(),xflt.max(),yflt.min(),yflt.max())

[ 3341.608  3910.253  4501.74  ...   207.812   231.688 23000.   ] [383937.29213972 383075.39740862 383561.3313806  ... 381774.93048964
 408644.3045959  325753.89200049] [3458836.09728581 3458402.22885706 3459283.67489885 ... 3444338.25324758
 3485639.08187888 3476716.32347716]
-500.0 25383.42 301395.4455435675 559009.3111848088 3416124.4850358246 3632469.490056862


/Users/shihao/anaconda3/envs/Python3.7/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/shihao/anaconda3/envs/Python3.7/lib/python3.7/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/Users/shihao/anaconda3/envs/Python3.7/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.i

In [4]:
#### predefine the grid networks     
zmin,zmax, xmin,xmax,ymin,ymax =  z.min(),z.max(),xflt.min(),xflt.max(),yflt.min(),yflt.max()
print(zmin,zmax, xmin,xmax,ymin,ymax )


-500.0 25383.42 301395.4455435675 559009.3111848088 3416124.4850358246 3632469.490056862


In [9]:
# generate GMsh model
# Helper function to return a node tag given two indices i and j:


In [25]:
# add faults surface loaded from stl  file
# e.g. wenchuan fault
import gmsh

ext =5e3
xmin,xmax,ymin,ymax =  xflt.min()-ext,xflt.max()+ext,yflt.min()-ext,yflt.max()+ext
zmin = -50.0e3


def generate_mesh_fault(MeshFile,stlFile):
    
    lc = 20.0e3
    N = 200
    PI = np.pi/180

    gmsh.initialize()
    gmsh.model.add("Wenchuanfault")

    r = 1000.0
    resolution = 1000.0

    ext = 50e3
    xmin,xmax,ymin,ymax =  xflt.min()-ext,xflt.max()+ext,yflt.min()-ext,yflt.max()+ext
    zmin = -50.0e3

    L, B, H = xmax-xmin, ymax-ymin, -zmin 
    x0, y0, z0 = xmin, ymin, zmin
    xc,yc,zc = (xmax+xmin)/2.0,(ymax+ymin)/2.0, -zmin/2.0
    
    # Step 1: Create the box geometry
    box_length = L
    box_width = B
    box_height = H

    # Add points defining the corners of the box
    p1 = gmsh.model.geo.addPoint(x0, y0, z0, lc)
    p2 = gmsh.model.geo.addPoint(x0+ box_length, y0, z0, lc)
    p3 = gmsh.model.geo.addPoint(x0+ box_length, y0+ box_width, z0, lc)
    p4 = gmsh.model.geo.addPoint(x0, y0+ box_width, z0, lc)
    p5 = gmsh.model.geo.addPoint(x0, y0, z0+ box_height, lc)
    p6 = gmsh.model.geo.addPoint(x0+box_length, y0, z0+box_height, lc)
    p7 = gmsh.model.geo.addPoint(x0+box_length, y0+ box_width, z0+box_height, lc)
    p8 = gmsh.model.geo.addPoint(x0, y0+box_width, z0+box_height, lc)

    # Add lines connecting the points to form the edges of the box
    l1 = gmsh.model.geo.addLine(p1, p2)
    l2 = gmsh.model.geo.addLine(p2, p3)
    l3 = gmsh.model.geo.addLine(p3, p4)
    l4 = gmsh.model.geo.addLine(p4, p1)
    
    l5 = gmsh.model.geo.addLine(p5, p6)
    l6 = gmsh.model.geo.addLine(p6, p7)
    l7 = gmsh.model.geo.addLine(p7, p8)
    l8 = gmsh.model.geo.addLine(p8, p5)
    
    l9 = gmsh.model.geo.addLine(p1, p5)
    l10 = gmsh.model.geo.addLine(p2, p6)
    l11 = gmsh.model.geo.addLine(p3, p7)
    l12 = gmsh.model.geo.addLine(p4, p8)
    
    gmsh.model.geo.synchronize()

    # Add curves defining the curves loops of the box
    cl1 = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])
    cl2 = gmsh.model.geo.addCurveLoop([l5, l6, l7, l8]) # free-surface
    cl3 = gmsh.model.geo.addCurveLoop([l1, l10, -l5, -l9])
    cl4 = gmsh.model.geo.addCurveLoop([l2, l11, -l6, -l10])
    cl5 = gmsh.model.geo.addCurveLoop([l3, l12, -l7, -l11])
    cl6 = gmsh.model.geo.addCurveLoop([l4, l9, -l8, -l12])
    gmsh.model.geo.synchronize()

    # Define the box surface by adding the surface loops
    s1 = gmsh.model.geo.addPlaneSurface([cl1])
    s2 = gmsh.model.geo.addPlaneSurface([cl2]) # free surface
    s3 = gmsh.model.geo.addPlaneSurface([cl3])
    s4 = gmsh.model.geo.addPlaneSurface([cl4])
    s5 = gmsh.model.geo.addPlaneSurface([cl5])
    s6 = gmsh.model.geo.addPlaneSurface([cl6])
    gmsh.model.geo.synchronize()

    # Step 2: Load the STL surface file
    gmsh.merge(stlFile)

    # Step 3: Create a compound entity containing both the box and the loaded STL surface
    # Step 4: Merge the compound entity
    
    sl1 = gmsh.model.geo.addSurfaceLoop([s1,s2, s3, s4, s5, s6])
    v1 = gmsh.model.geo.addVolume([sl1])
    gmsh.model.geo.synchronize()
    

#     merged_entities = gmsh.model.geo.merge()
    
#     box_tag = merged_entities[0][1]  # Assuming the box is the first entity in the merged entities list
#     box_group_tag = 1  # Assigning physical group tag 1 to the box
    
    gmsh.model.addPhysicalGroup(3, [v1], 1 )

    # mesh size
    distance = gmsh.model.mesh.field.add("Distance")
    gmsh.model.mesh.field.setNumbers(distance, "FacesList", [7])
    
    threshold = gmsh.model.mesh.field.add("Threshold")
    gmsh.model.mesh.field.setNumber(threshold, "IField", distance)
    gmsh.model.mesh.field.setNumber(threshold, "LcMin", resolution)
    gmsh.model.mesh.field.setNumber(threshold, "LcMax", 10*resolution)
    gmsh.model.mesh.field.setNumber(threshold, "DistMin", 0.5*r)
    gmsh.model.mesh.field.setNumber(threshold, "DistMax", r)
   
    minimum = gmsh.model.mesh.field.add("Min")
    gmsh.model.mesh.field.setNumbers(
        minimum, "FieldsList", [threshold])
    gmsh.model.mesh.field.setAsBackgroundMesh(minimum)
    # Generate the mesh
    
    gmsh.model.geo.synchronize()
    gmsh.model.mesh.generate(3)
    
    # asign physical group in gmsh
    faces = gmsh.model.getEntities(dim=2)
    print('all faces:',faces)

    absorb_end = [1,3,4,5,6]
    faults_end = [7]

    print("dynamic BC:", faults_end)
    print("absorbing BC:", absorb_end)
            
    # add group 103 and 105 for test
    gmsh.model.addPhysicalGroup(2, absorb_end,105) 
    gmsh.model.addPhysicalGroup(2, faults_end,103) 
    gmsh.model.addPhysicalGroup(2, [2],101) 

    # Optionally, visualize the mesh
#     gmsh.fltk.run()
    gmsh.write(MeshFile+'.msh2') # type 2 Gmsh file  

    # Finalize Gmsh
    gmsh.finalize()

In [26]:
# change the dir_to_root_folder

rootfolder = '/Users/shihao/Documents/SeisSol/wenchuan/'
    
MeshFile = rootfolder + 'WenchuanFault2'
stlFile = rootfolder + 'CDYingxiu.stl'

generate_mesh_fault(MeshFile,stlFile)


Info    : Reading '/Users/shihao/Documents/SeisSol/wenchuan/CDYingxiu.stl'...
Info    : 29474 facets in solid 0 MATLAB_1
Info    : Done reading '/Users/shihao/Documents/SeisSol/wenchuan/CDYingxiu.stl'
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 50%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.0039805s, CPU 0.004583s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing s

In [139]:
# gmsh.finalize()

In [5]:
# generate a xdmf file for Paraview
import meshio

# Load the .msh file
def convert_mesh(MeshFile):
    
    mesh = meshio.read(
    MeshFile+ '.msh2',  # string, os.PathLike, or a buffer/open file
    file_format="gmsh",  # optional if filename is a path; inferred from extension
    )

    meshio.write(
    MeshFile+".vtk",  # str, os.PathLike, or buffer/ open file
    mesh,
    file_format="vtk",  # optional if first argument is a path; inferred from extension
    )
    

In [6]:
convert_mesh(MeshFile)

NameError: name 'MeshFile' is not defined